In [1]:
!pip install tensorflow


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at:', device_name)


Found GPU at: /device:GPU:0


In [3]:
import pandas as pd
from transformers import BertTokenizer

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
from transformers import AutoTokenizer
from datasets import Dataset

In [7]:
df_train = pd.read_csv("/content/drive/My Drive/LLM_project/clean_dataset/clean_train.csv")
df_test = pd.read_csv("/content/drive/My Drive/LLM_project/clean_dataset/clean_test.csv")
df_unsupervised = pd.read_csv("/content/drive/My Drive/LLM_project/clean_dataset/unsupervised_clean.csv")


In [8]:
df_train.head()

,text,label,clean_text
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,rented curiousyellow video store controversy s...
1,"""I Am Curious: Yellow"" is a risible and preten...",0,curious yellow risible pretentious steaming pi...
2,If only to avoid making this type of film in t...,0,avoid making type film future film interesting...
3,This film was probably inspired by Godard's Ma...,0,film probably inspired godards masculin fémini...
4,"Oh, brother...after hearing about this ridicul...",0,oh brotherafter hearing ridiculous film umptee...


In [9]:
df_unsupervised.head()

,text,label,clean_text
0,This is just a precious little diamond. The pl...,-1,precious little diamond play script excellent ...
1,When I say this is my favourite film of all ti...,-1,say favourite film time comment taken lightly ...
2,I saw this movie because I am a huge fan of th...,-1,saw movie huge fan tv series name starring roy...
3,Being that the only foreign films I usually li...,-1,foreign films usually like star japanese perso...
4,After seeing Point of No Return (a great movie...,-1,seeing point return great movie told original ...


In [10]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model configuration and modify dropout probabilities
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=2,
    hidden_dropout_prob=0.3,                # Increase dropout to 0.2
    attention_probs_dropout_prob=0.2       # Adjust for attention layers
)

# Initialize the model with the updated configuration
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config
)


def tokenize_data(examples):
    return tokenizer(examples["clean_text"], padding=True, truncation=True, max_length=512)

model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2) #2 labels for the st

def tokenize_data(examples):
    return tokenizer(examples["clean_text"], padding=True, truncation=True, max_length=512)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Let's say the columns are 'clean_text' for reviews and 'label' for sentiment
texts = df_train['clean_text'].tolist()  # The list of reviews (texts)
labels = df_train['label'].tolist()  # The list of sentiment labels (0 or 1)

In [12]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.1, random_state=42
)


In [13]:
# tokenize the data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

In [14]:
# Convert tokenized data into Hugging Face Dataset format
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

In [15]:
import torch

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


In [16]:
from transformers import TrainingArguments


training_args = TrainingArguments(
    output_dir="./results",           # Output directory for saved models
    num_train_epochs=3,               # Number of epochs
    per_device_train_batch_size=16,    # Batch size per device during training
    per_device_eval_batch_size=16,    # Batch size for evaluation
    warmup_steps=500,                 # Number of warmup steps
    weight_decay=0.1,                 # Strength of weight decay
    learning_rate=5e-6,              # Lowered learning rate for stability
    logging_dir="./logs",             # Directory for storing logs
    logging_steps=10,                 # Log every 10 steps
    evaluation_strategy="epoch",      # Evaluate every epoch
    save_strategy="epoch",            # Save model every epoch
    load_best_model_at_end=True,      # Load the best model at the end
    metric_for_best_model="f1",       # Use F1-score to track best model
    greater_is_better=True,           # Higher F1-score is better
    lr_scheduler_type="cosine"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Define the function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)  # Convert logits to predicted labels
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Now, initialize the Trainer with compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Now it's defined
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)


<ipython-input-17-9b4fca5deaff>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
import wandb
wandb.init(project="your_project_name")  # Initialize the wandb logging


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jhovanisabuchi (jhovanisabuchi-lighthouse-labs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [19]:
# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_model")


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.276100,0.283773,0.882800,0.882924,0.882800,0.882726
2,0.296600,0.270540,0.893200,0.893424,0.893200,0.893116
3,0.204500,0.278982,0.897600,0.897739,0.897600,0.897623


In [21]:
test_texts = df_test['clean_text'].tolist()
test_labels = df_test['label'].tolist()

# Tokenize the test data
test_encodings = tokenizer(
    test_texts,
    truncation=True,
    padding=True,
    max_length=512
)

# Create the test dataset
test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels
})

# Format the test dataset for PyTorch
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


In [22]:
test_results = trainer.evaluate(test_dataset)
print(test_results)


{'eval_loss': 0.26839572191238403, 'eval_accuracy': 0.90056, 'eval_precision': 0.900566409062545, 'eval_recall': 0.90056, 'eval_f1': 0.900559602238409, 'eval_runtime': 82.3108, 'eval_samples_per_second': 303.727, 'eval_steps_per_second': 18.989, 'epoch': 3.0}


In [25]:
from google.colab import drive
drive.mount('/content/drive')

trainer.save_model("/content/drive/My Drive/LLM_project/model")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
!ls "/content/drive/My Drive/LLM_project/model"


In [26]:
!ls "/content/drive/My Drive/LLM_project/model"


config.json	   special_tokens_map.json  tokenizer.json     vocab.txt
model.safetensors  tokenizer_config.json    training_args.bin
